In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
# Skriptas Zive įrašų talpykloje kiekvienam pacientui atrenka iki n_max įrašų su daugiausiai
# turinčiais ekstrasistolių, sudaro jų sąrašą ir patalpina į atskirą folderį.
# 
# Planas:
# sukuriame tuščią dataframe df_list_selected, iš kurio darysime naują list.json
# Nuskaitome iš bendros talpyklos list.json
# pasidarom sąrašą userId:recordId1, recordId2,.... 
# ciklas per visus pacientus userId:
#   suformuojame sąrašą recordId: ekstrasistolių skaičius ir surušiuojame pagal ekstrasistolių
#   skaičių mažėjančia tvarka
#   Surikiavus įrašus, įrašus atrenkame tokiu būdu:
# 		a. Atrenkame pirmiausiai tuos įrašus, kurie turi daugiausiai ekstrasistolių. 
#       Jei visi įrašai turi ekstrasistolių, tai atrenkame pirmus 10 (iš surikiuotų įrašų pagal ekstrasistolių kiekį).
#       Jei ne visi įrašai turi ekstrasistoles, tai atrenkame, tik tuos įrašus, kurie turi ekstrasistoles
#       (bet ne daugiau 10 įrašų), ir jei įrašų atrinkta ne daugiau 9, pridedame dar vieną įrašą be ekstrasistolių.
# 		b. Jei pacientas ekstrasistolių neturi, tai atrenkame ne daugiau vieno (pirmo) to paciento įrašo

import os, shutil, sys
from pathlib import Path
import pandas as pd

import json
from zive_util_vu import create_dir

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU_TST'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder, 'records')

# Nuoroda į aplanką su atrinktų EKG duomenų rinkiniu
db_path_selected = Path(Duomenu_aplankas, db_folder, 'records_selected')

# Atrenkamų iš vieno paciento įrašų max skaičius
max_count = 10


# ////////////////////////////////////////////////////////////////////////////////////

print("\nSkriptas zive įrašų atrankai klasifikatoriaus mokymui\n")

# Sukūriame aplanką EKG sekų įrašymui
create_dir(db_path_selected)

# paruošiame dataframe atrinktų įrašų sąrašui

df_list_selected = pd.DataFrame({
                        'file_name': pd.Series(dtype='str'),
                        'userId': pd.Series(dtype='str'),
                        'recordingId': pd.Series(dtype='str'),
                        'N': pd.Series(dtype='int'),
                        'S': pd.Series(dtype='int'),
                        'V': pd.Series(dtype='int'),
                        'U': pd.Series(dtype='int'),
                        'incl': pd.Series(dtype='int'), 
                        'flag': pd.Series(dtype='int'),
                        'recorded_at':pd.Series(dtype='str'), 
                        'E': pd.Series(dtype='int')
                        })

pd.set_option("display.max_rows", 6000, "display.max_columns", 18)
pd.set_option('display.width', 1000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(db_path, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# Suformuojame sąraše naują kolonėlę su ekstrasistolių kiekiu
df_list['E'] = df_list['S'] + df_list['V']

# Sugrupuojame įrašus į to paties paciento grupes
df_grouped = df_list.groupby(['userId'])
# print(df_grouped)

# Išrenkame įrašus su didžiausiu kiekiu ekstrasistolių, formuojame sąrašą,
# perrašome įrašus į atskirą folderį

for name, group in df_grouped:
    # print(name)
    sorted_group = group.sort_values(by='E', ascending=False)
    records_count = len(sorted_group)
    if (records_count > max_count):
        records_count = max_count
    # print(f'{max_count=}')
    
    count = 0    
    for idx, df_row in sorted_group.iterrows():
        if (df_row['E'] == 0):
            df_list_selected = df_list_selected.append(df_row, ignore_index=True)
            break 
        else:
            count +=1
            if  (count <= max_count and df_row['E'] != 0):
                # print(df_row['userId'], df_row['recordingId'], df_row['E'])
                df_list_selected = df_list_selected.append(df_row, ignore_index=True)
            else:
                if (df_row['E'] == 0):
                    df_list_selected = df_list_selected.append(df_row, ignore_index=True)
                break

# Perrašome atrinktus įrašus į diską  - čia reiktų paversti duomenis į npy formatą //////////////////////////////////
for idx, row in df_list_selected.iterrows():
    file_name = row['file_name']
    src = Path(db_path, file_name)
    dst = Path(db_path_selected, file_name)
    shutil.copy2(src, dst)
    src = Path(db_path, file_name + '.json')
    dst = Path(db_path_selected, file_name + '.json')
    # shutil.copyfile(src, dst)
    shutil.copy2(src, dst)
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list_selected.groupby(['userId'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list_selected['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list_selected)}')
print(f'\nAtrinktų įrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['incl'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

selected_columns = result[['E']]
result_new = selected_columns.copy()
print(result_new)

sorted_group = result_new.sort_values(by='E', ascending=False)
print(sorted_group)

print("\nAtrinktų EKG įrašų sąrašas\n")
df_list_selected.drop(labels=['E'], axis=1, inplace=True)

# Įrašome naujai suformuotą sąrašą į diską
file_path = Path(db_path_selected,'list.json')       
df_list_selected.to_json(file_path, orient = 'table', index=False)
print(f'\nFailų sąrašas įrašytas:  {file_path}')




OS in my system :  win32

Skriptas zive įrašų atrankai klasifikatoriaus mokymui

Directory 'D:\DI\Data\MIT&ZIVE\VU\DUOM_VU_TST\records_selected' already exists

Viso pacientų: 12  EKG įrašų: 57

Atrinktų įrašų pasiskirstymas per pacientus
                             N   S    V  U  flag    E  count
userID                                                      
6034c808d6c2740008035ede  1621   6    0  0     0    6      2
60a917b354352a3df86dc1f2  9970  16   10  5     0   26      6
60e1d80f93b55b41529e9eaa  7238  11   72  0     0   83      7
613b1bc63d08d41309cdc8f1  1186   0    1  0     0    1      2
613b1c013d08d44862cdc8f2  1238   2    0  0     0    2      2
613b1c6f3d08d4370acdc8f3  5658  23    2  0     0   25     10
613b1c9c3d08d43181cdc8f4  1177   0    1  0     0    1      2
613b1cc33d08d4c0d5cdc8f5  1260   1    0  0     0    1      2
613b1d0c3d08d413ffcdc8f6  7930  26  145  0     0  171     10
613b1d673d08d4d1f3cdc8f8  7784  91    0  5     0   91     10
613b1d903d08d4df57cdc8f9  151